# Writing Files, Inventorying Files

This notebook goes through the basics of writing files. We look through one basic example and one that extracts specific information from one file then writes it to a new file. After that, we look at a few modules that will help us to build an inventory of basic system information including filenames, locations (paths), and sizes. Once we identify this information we can use it to create an inventory manifest. 

## Python libraries/modules used

* `os` module ([docs](https://docs.python.org/3/library/os.html)) for navigating and interacting with the file system
* `os.path` module ([docs](https://docs.python.org/3/library/os.path.html)) for working with paths - note this is a submodule of `os`, so when you import `os`, you also have access to all of these functions
* `shutil` module ([docs](https://docs.python.org/3/library/shutil.html)) for copying, moving, removing


## Key points / Questions

* How can I interact with the files on my computer using python?
* How can I work with file paths in various operating systems?
* How do I move around the file system in Python? 
* How can I find out the current location on the file system, and how can I look in other locations, using Python?
* How can I identify file information and metadata using Python?

First, let's look at writing files. 

## Writing Files

The basic function for writing files is the `write()` function. This can be used to write contents from the argument or 
to write multi-line content. Unlike in other environments like the GUI or shell, where the open command is often assumed, 
you may need to `open()` and then `close()` files when working in python. You cannot write to a file that is not known and opened, and a file that is not closed may be corrupted. 

Fortunately, we can usually use the contexual opener:

```python
with open(file, 'w') as f:
    ```

This will automatically close the file when the loop completes. The `w` argument indicates that the file is opened in "write" mode. If the file doesn't exist, the file will be written. 

In [1]:
# Basic use of open() and write()

line = 'Believe that life is worth living, and your belief will help create the fact.'
# Credit William James https://en.wikiquote.org/wiki/William_James

fout = open('quote-output.txt', 'w')

fout.write(line)

fout.close()

In [2]:
# use the with open() syntax to check if the file is there

with open('quote-output.txt', 'r') as f:
    print(f.read())

Believe that life is worth living, and your belief will help create the fact.


## OS Navigation and File Paths

Now, let's look at moving around and managing file paths. This uses the `os` and `os.path` libraries. Importing `os` will import all of the `os.path` functions as well. We can use the `os` module to interact with the operating system. For example, 
functions similar to file navigation on the terminal.

In [3]:
import os

The `getcwd()` function is like `pwd`:

In [4]:
os.getcwd()

'/Users/jajohnst/Desktop/networked-services-labs-2022'

The `chdir()` allows you to move around, like `cd`:

In [5]:
os.chdir('data')

In [6]:
os.getcwd()

'/Users/jajohnst/Desktop/networked-services-labs-2022/data'

In [7]:
os.chdir('..')

In [8]:
os.getcwd()

'/Users/jajohnst/Desktop/networked-services-labs-2022'

The path module allows building up paths using multiple arguments by combining them together as arguments in the `os.path.join()` function. This can also be assigned to a variable, such as `path_to_file` below:

In [9]:
file = 'mbox-short.txt'

path_to_file = os.path.join('data','emails',file)

print(path_to_file)

data/emails/mbox-short.txt


## Identifying parts of the path

The path module also helps to split up paths. For example, to extract the directory location of a file, the name of a file, and even the extension of a file. These functions do this:

* `os.path.basename()` returns what usually is considered the filename. Technically whatever is at the last position of the path after the final slash or path separator character
* `os.path.dirname()` returns a string of everything before the last separator
* `os.path.splitext()` returns a tuple containing the basename and the extension of the file. This assumes that the file extension is indicated by a dot before the last bit of the file. 

In [10]:
path_to_file

'data/emails/mbox-short.txt'

In [11]:
dirName = os.path.dirname(path_to_file)
baseName = os.path.basename(path_to_file)
fExtension = os.path.splitext(path_to_file)

print(dirName,'\n',baseName,'\n',fExtension)

data/emails 
 mbox-short.txt 
 ('data/emails/mbox-short', '.txt')


In [12]:
# to isolate the file extension portion:
fExtension[1]

'.txt'

## Absolute & Relative Paths

* `os.path.abspath()` returns the absolute path as a string
* `os.path.isabs()` returns a `True` or `False` depending on whether argument it receives is an absolute path or not
* `os.path.relpath()` returns a string of the relative path

As in the shell terminal, we can refer to the current directory with the `.` file, and the parent directory with two dots like `..`.

In [13]:
os.path.abspath('.')

'/Users/jajohnst/Desktop/networked-services-labs-2022'

In [14]:
cur_loc = os.getcwd()

os.path.isabs(cur_loc)

True

Note how it is possible to "ask" for the relative path to your current location, and you should receive the dot directory `.`.

In [15]:
# what is the relative path to the current location?
os.path.relpath(os.getcwd())

'.'

The relative path can also be used to return what the path to another location is, relative to the current location:

In [16]:
os.path.relpath('data','webfiles-samples')

'../data'

## Getting file information (i.e., metadataz)

Finally, `os.path` allows us to also query the location (if it's a file) to show things like size, date modified, etc. The basic things, like the size of the file are easier to get to:

* `os.path.getsize()` returns the file size in bytes. 

Other information, like modification time, is contained in a property called `stat_result`. This is accessed via the `stat` method and is a part of file path objects. (Here is where we may need to test if something is or is not a file.) In brief, stat contains things like:

* `st_size` also contains the size in bytes
* `st_mtime` indicates the modification time in a value that indicates time according to the computer's operating system. Note that Unix and Windows systems may calculate time differently, and these may or may not have reliable information. In other cases, the time may be called modify time but the system records something else, like the last time the file was accessed (vs the last time it was saved/written), or the time it was created. This is, unfortunately (for those of us interested in these kinds of things), very difficult to track across systems, so always take this under advisement. 



In [17]:
path_to_file

'data/emails/mbox-short.txt'

In [18]:
os.stat(path_to_file)

os.stat_result(st_mode=33188, st_ino=4550330, st_dev=16777234, st_nlink=1, st_uid=504, st_gid=20, st_size=94625, st_atime=1663731271, st_mtime=1663731214, st_ctime=1663731256)

This can be assigned to a variable, and the various elements of the result object called using dot notation. For example, to make a list of the file size and modify time:

In [19]:
file_stats = os.stat(path_to_file)

file_info = {
    'size' : file_stats.st_size,
    'modify_time' : file_stats.st_mtime
}

print(file_info)

{'size': 94625, 'modify_time': 1663731214.6876292}


Now, building these all together, we have a way of extracting the file metadata! :tada:

In [20]:
file_stats = os.stat(path_to_file)

file_info = {
    'absPath' : os.path.abspath(path_to_file),
    'directory_location' : os.path.dirname(path_to_file),
    'name' : os.path.basename(path_to_file),
    'extension' : os.path.splitext(path_to_file)[1],
    'size': os.path.getsize(path_to_file),
    'modify_time' : file_stats.st_mtime
}

print(file_info)

{'absPath': '/Users/jajohnst/Desktop/networked-services-labs-2022/data/emails/mbox-short.txt', 'directory_location': 'data/emails', 'name': 'mbox-short.txt', 'extension': '.txt', 'size': 94625, 'modify_time': 1663731214.6876292}


## Interacting with files and paths

We can also extract information from a file then reuse that in another file. 
For example, we could extract the email addresses from `mbox-short.txt` and create
an address book file:

In [21]:
# set up a file name for a file to create
fout = 'email-list.txt'

#establish a list to record emails as they are identified
emails = []

# open the source file to extract emails
with open(path_to_file, 'r') as f:
    for line in f:
        if line.startswith('From:'):
            email = line[6:].rstrip()
            if email not in emails:
                emails.append(email)
print(emails, '\n\n')

# open another file in write mode to write the emails.
with open(fout, 'w') as f:
    for email in emails:
        f.write(email + '\n')

print(open(fout).read())

['stephen.marquard@uct.ac.za', 'louis@media.berkeley.edu', 'zqian@umich.edu', 'rjlowe@iupui.edu', 'cwen@iupui.edu', 'gsilver@umich.edu', 'wagnermr@iupui.edu', 'antranig@caret.cam.ac.uk', 'gopal.ramasammycook@gmail.com', 'david.horwitz@uct.ac.za', 'ray@media.berkeley.edu'] 


stephen.marquard@uct.ac.za
louis@media.berkeley.edu
zqian@umich.edu
rjlowe@iupui.edu
cwen@iupui.edu
gsilver@umich.edu
wagnermr@iupui.edu
antranig@caret.cam.ac.uk
gopal.ramasammycook@gmail.com
david.horwitz@uct.ac.za
ray@media.berkeley.edu



## Inventorying Files

For this activity, we are going to expand the use of modules that allow us to interact with the file system. These should be somewhat familiar after we have already looked into basic shell commands.

* `os` assists in using aspects of the operating system, in this case particularly file information and paths. See https://docs.python.org/3/library/os.html; 
* `os.path` is often called by itself and allows us to interact with file path and directory information. See https://docs.python.org/3/library/os.path.html#module-os.path. 
* `shutil` allows to access some shell utilities, like move, copy, rename, delete. See https://docs.python.org/3/library/shutil.html?highlight=shutils.

We will also use the `csv` module since it will help us to write the information that we gather to a structured data file that can later be opened in Excel or other spreadsheet applications. See https://docs.python.org/3/library/csv.html

In [22]:
# if you haven't yet imported the modules, run this cell
import os
# if you only want to access certain elements of the modules, or to alias them to simpler calls, run this line
#from os.path import join, getsize, getctime
# activities to write out the files to a CSV will use the CSV module
import csv

Once we know what we want in the csv, how do we get that information? We can use the `os` module to get file information. We will use the `os.walk` function to "walk" over the file tree, identify folder lists, paths, and filenames.  

In [23]:
walk_this_directory = os.path.join('data','webfiles-samples')

print(walk_this_directory)

data/webfiles-samples


### Using os.listdir()

We can generate a list of the files in the directory using the `os.listdir()` function. This list will include the file names for all the files in the directory. 

In [24]:
dir_list = os.listdir(walk_this_directory)

print(dir_list)

['video', '.DS_Store', 'pdf', 'image', 'audio', '.ipynb_checkpoints', 'web-files-small-metadata.csv', 'presentation']


Let's use the `listdir()` function to create manifest of the files in the `pdf` directory:

In [25]:
# create the list
pdf_dir_list = os.listdir(os.path.join(walk_this_directory, 'pdf'))

# set up a file name for a file to create
fout = 'pdf-file-list.txt'

# open outfile in write mode to write the filenames
with open(fout, 'w') as f:
    for filename in pdf_dir_list:
        f.write(filename)
        f.write('\n')
print('wrote',fout)

wrote pdf-file-list.txt


To view the contents of a file quickly, it's possible to use the `.read()` like `cat` on the terminal.

In [26]:
print(open(fout).read())

01-1480.pdf
file.pdf
Chapter03.pdf
PFCHEJ.pdf
HR2021 commtext.pdf



We could, for example, use this to calculate the total size of the files in the directory: 

In [27]:
total_size = 0

# set up a path to the directory
pdf_directory = os.path.join(walk_this_directory, 'pdf')

# use listdir() to loop through the directory and tally up the file sizes:
for pdf in os.listdir(pdf_directory):
    total_size = total_size + os.path.getsize(os.path.join(walk_this_directory, 'pdf', pdf))

print('The directory pdf has this many bytes:')
print(total_size)

The directory pdf has this many bytes:
149427


Note: the `listdir()` function creates a list. It's helpful to use when creating a list of the files, but for more complex operations, there are additional options that allow us to iterate through, or loop through the contents of a directory in more useful ways. For example, note the somewhat tedious reiteration of `os.path.join()` when going through the list of files in the for loop in the above block.

### Using os.scandir()

The previous steps are a good way to record a simple list of files in a directory, but there is still a lot of information from the filepath information. For example, what are the abosolute paths to the files? How big are the files? That information is stored in the filesystem, but we do not get it using the listdir() method. 

We can use the `scandir()` function to get a list that we can iterate through. For example, we may want to check whether items are recognized as files by the system. Other functions, like `os.is_file()`, will evaluate whether the item is a file. We can use this function to create data that we can iterate through using a `with` ... `as` construction, like we have seen in opening files.

In [28]:
# recall, we have set walk_this_directory to the data/webfiles-small directory 
walk_this_directory

'data/webfiles-samples'

In [29]:
item_count = 0

with os.scandir(walk_this_directory) as items_list:    
    for entry in items_list:
        item_count += 1
        print('Looking at:',entry)
        if entry.is_dir():
            file_list = os.listdir(os.path.join(entry))
            print('This is a directory and contains',len(file_list),'files (',file_list,').')
        if entry.is_file():
            print('This is a file named',entry.name,'that takes up',os.path.getsize(entry),'bytes')

print('Scanned',item_count,'items.')

Looking at: <DirEntry 'video'>
This is a directory and contains 4 files ( ['vlwhcssc.asx', '04-04-21full.asf', 'glmp_cig.EQ.wm.p20.t12z', 'oct17cc.asx'] ).
Looking at: <DirEntry '.DS_Store'>
This is a file named .DS_Store that takes up 6148 bytes
Looking at: <DirEntry 'pdf'>
This is a directory and contains 5 files ( ['01-1480.pdf', 'file.pdf', 'Chapter03.pdf', 'PFCHEJ.pdf', 'HR2021 commtext.pdf'] ).
Looking at: <DirEntry 'image'>
This is a directory and contains 5 files ( ['13080t.jpg', 'orca.via_.moc_.noaa_.jpg', 'k7989-7x.jpg', 'm237a2f.gif', '1005107061.tif'] ).
Looking at: <DirEntry 'audio'>
This is a directory and contains 4 files ( ['11-3250JohnsonvFolinoEtAl.wma', 'NEWSLINE_802AF71F439D401585C6FCB02F358307.mp3', 'mj_telework_exchange_final_100710.mp3', '000727.ram'] ).
Looking at: <DirEntry '.ipynb_checkpoints'>
This is a directory and contains 1 files ( ['web-files-small-metadata-checkpoint.csv'] ).
Looking at: <DirEntry 'web-files-small-metadata.csv'>
This is a file named web

### Using os.walk()

The `os.walk()` function allows us to do a more complex mapping of the directory. This function can be used to create a "tuple" &ndash; a special datatype that creates a small, unmutable set that we can reuse &ndash; and we can store that information to derive foldernames and paths to individual files. 

In [30]:
for folderName, subfolders, filenames in os.walk(walk_this_directory):
    # see what this produces
    print('folderName is a type',type(folderName),
          '\nsubfolders is a type',type(subfolders),
          '\nfilenames is a type',type(filenames))

    ## so, this is a series of nested loops, 
    ## the top level produces a string for the folder name, 
    ## and the secondary levels create lists of the contained folders and files 

folderName is a type <class 'str'> 
subfolders is a type <class 'list'> 
filenames is a type <class 'list'>
folderName is a type <class 'str'> 
subfolders is a type <class 'list'> 
filenames is a type <class 'list'>
folderName is a type <class 'str'> 
subfolders is a type <class 'list'> 
filenames is a type <class 'list'>
folderName is a type <class 'str'> 
subfolders is a type <class 'list'> 
filenames is a type <class 'list'>
folderName is a type <class 'str'> 
subfolders is a type <class 'list'> 
filenames is a type <class 'list'>
folderName is a type <class 'str'> 
subfolders is a type <class 'list'> 
filenames is a type <class 'list'>
folderName is a type <class 'str'> 
subfolders is a type <class 'list'> 
filenames is a type <class 'list'>


In [31]:
# get information about how many files are in each directory and how much space they take up
for FolderPaths, SubfolderNames, filenames in os.walk(walk_this_directory):
    print(FolderPaths, "consumes", end=" ")
    print(sum(os.path.getsize(os.path.join(FolderPaths, name)) for name in filenames), end=" ")
    print("bytes in", len(filenames), "non-directory files")


data/webfiles-samples consumes 15217 bytes in 2 non-directory files
data/webfiles-samples/video consumes 115695 bytes in 4 non-directory files
data/webfiles-samples/pdf consumes 149427 bytes in 5 non-directory files
data/webfiles-samples/image consumes 497284 bytes in 5 non-directory files
data/webfiles-samples/audio consumes 25856261 bytes in 4 non-directory files
data/webfiles-samples/.ipynb_checkpoints consumes 9069 bytes in 1 non-directory files
data/webfiles-samples/presentation consumes 289792 bytes in 3 non-directory files


In [32]:
for folderName, subfolders, filenames in os.walk(walk_this_directory):
    print('Current folder:',folderName)
    
    for subfolder in subfolders:
        print('Parent folder:',folderName,'; subfolder:',subfolder)
        
    for filename in filenames:
        print('The file', filename, 
              '\n    This is the folder:', folderName,
             '\n    The filepath is:',os.path.join(folderName, filename))
    
    print('\n')

    ## Note that this does not record hidden items like . and ..

Current folder: data/webfiles-samples
Parent folder: data/webfiles-samples ; subfolder: video
Parent folder: data/webfiles-samples ; subfolder: pdf
Parent folder: data/webfiles-samples ; subfolder: image
Parent folder: data/webfiles-samples ; subfolder: audio
Parent folder: data/webfiles-samples ; subfolder: .ipynb_checkpoints
Parent folder: data/webfiles-samples ; subfolder: presentation
The file .DS_Store 
    This is the folder: data/webfiles-samples 
    The filepath is: data/webfiles-samples/.DS_Store
The file web-files-small-metadata.csv 
    This is the folder: data/webfiles-samples 
    The filepath is: data/webfiles-samples/web-files-small-metadata.csv


Current folder: data/webfiles-samples/video
The file vlwhcssc.asx 
    This is the folder: data/webfiles-samples/video 
    The filepath is: data/webfiles-samples/video/vlwhcssc.asx
The file 04-04-21full.asf 
    This is the folder: data/webfiles-samples/video 
    The filepath is: data/webfiles-samples/video/04-04-21full.asf


In [33]:
file_count = 0

# get information about each of the files
for folderName, subfolders, filenames in os.walk(walk_this_directory):
    
    for filename in filenames:
        file_count += 1
        filename = filename 
        folder = folderName
        path = os.path.join(folderName, filename)
        absolutePath = os.path.abspath(filename)
        size = os.path.getsize(path)
        print('Found:', filename, folder, path, size,'\n',absolutePath)
print(f'Counted {file_count} files')
    ## Note that this does not record hidden items like . and ..

Found: .DS_Store data/webfiles-samples data/webfiles-samples/.DS_Store 6148 
 /Users/jajohnst/Desktop/networked-services-labs-2022/.DS_Store
Found: web-files-small-metadata.csv data/webfiles-samples data/webfiles-samples/web-files-small-metadata.csv 9069 
 /Users/jajohnst/Desktop/networked-services-labs-2022/web-files-small-metadata.csv
Found: vlwhcssc.asx data/webfiles-samples/video data/webfiles-samples/video/vlwhcssc.asx 356 
 /Users/jajohnst/Desktop/networked-services-labs-2022/vlwhcssc.asx
Found: 04-04-21full.asf data/webfiles-samples/video data/webfiles-samples/video/04-04-21full.asf 98 
 /Users/jajohnst/Desktop/networked-services-labs-2022/04-04-21full.asf
Found: glmp_cig.EQ.wm.p20.t12z data/webfiles-samples/video data/webfiles-samples/video/glmp_cig.EQ.wm.p20.t12z 8296 
 /Users/jajohnst/Desktop/networked-services-labs-2022/glmp_cig.EQ.wm.p20.t12z
Found: oct17cc.asx data/webfiles-samples/video data/webfiles-samples/video/oct17cc.asx 106945 
 /Users/jajohnst/Desktop/networked-ser

In [34]:
## get information about each of the files

# first set some counters
fileCount = 0

# and a list fileInfo to hold the information about the file, 
# and another list manifestInfo to hold information about all of the files
fileInfo = list()
manifestInfo = list()

for folderName, subfolders, filenames in os.walk(walk_this_directory):    
    for filename in filenames:
        fileCount += 1
        index = fileCount
        filename = filename 
        folder = folderName
        path = os.path.join(folderName, filename)
        size = os.path.getsize(path)
#        print('Found:', filename, folder, path, size)

        fileInfo = [
            index,
            filename,
            folder,
            path,
            size
            ]
        manifestInfo.append(fileInfo)

print('Found',len(manifestInfo),'items.\n\n',manifestInfo[:2])

Found 24 items.

 [[1, '.DS_Store', 'data/webfiles-samples', 'data/webfiles-samples/.DS_Store', 6148], [2, 'web-files-small-metadata.csv', 'data/webfiles-samples', 'data/webfiles-samples/web-files-small-metadata.csv', 9069]]


There are additional things that the `os` module can tell us. So let's take a moment to pause here to explore some of its additional uses. 

Now that we can handle file paths, we may want additional information about the files that we are taking a look at. 
Our goal is to create a script that inventories a specific directory and records metadata about the files in that directory.
Using `os.path`, it is possible to get the size of the file (bytes), well as additional path information, such as the full path to the file (aka, the "absolute path"), and the "base" name of the file (just the name of the file), as well as the file extension. 

To retrieve this metadata:

* `os.path.abspath(path)` returns a string that has the absolute path to the requested argument; this can also be used to convert a relative path to an absolute path
* `os.path.relpath(path, start)` returns a string of the relative path to the given argument "path," and you can specify where to "start", that is the location. 
* `os.path.getsize(path)` returns the size of the requested file location in bytes, note that this may require an absolute path argument, so as illustrated below you may need to create the full path
* `os.path.splitext(file)` returns a tuple with the first part of the filename as the first element and the the extension of the file as the second element)
* `os.path.getmtime(file)` returns a timestamp with the most recent modification time recorded; this should work on both MAC and Windows platforms, but some file metadata will be specific to your operating system; the returned value will be an era number, and to 'decode' the time will require a time conversion, which we will cover later

## Getting file hashes - TODO

There is an additional Python library that can help us create fixity information. The `hashlib` library can craete many different types of has digests, including MD5, SHA256, and SHA512.

TODO: As of September, this section still needs to be refined, so for the moment do not use.

In [44]:
import hashlib

def create_md5_digest(file):
    '''This is a short function to create a string of an md5 checksum, which can be used to check data integrity.
    
    The accepted file value should be a string representing a valid path.'''
    m = hashlib.md5(file.encode())
    return m.hexdigest()

In [ ]:
for file in os.scandir(os)

In [54]:
# for example, look at the pdfs

for file in os.scandir(os.path.join(walk_this_directory, 'pdf')):
    print(file,type(file))
    if os.path.isfile(file):
        path = os.path.join(walk_this_directory, 'pdf', file.name)
        print(path)
        md5digest = create_md5_digest(os.path.join(walk_this_directory, 'pdf', file.name))
        md5digest2 = create_md5_digest(file.name)
        print(file,':',md5digest, f'\ndigest2({file.name}):',md5digest2) # the encode here is to ensure that the file is treated as a file, not a string of characters. 
    else:
        print(f'ERROR: No file found at {file}.')

<DirEntry '01-1480.pdf'> <class 'posix.DirEntry'>
data/webfiles-samples/pdf/01-1480.pdf
<DirEntry '01-1480.pdf'> : cc188a5a6db69f970a98f8ea63ff61e7 
digest2(01-1480.pdf): 334ba0175117dad43b3608746fcccf7d
<DirEntry 'file.pdf'> <class 'posix.DirEntry'>
data/webfiles-samples/pdf/file.pdf
<DirEntry 'file.pdf'> : e1225c841c76e86a6f1ff96b04d15423 
digest2(file.pdf): dd35524cb5c8998dfc7eee0a13d0fcb1
<DirEntry 'Chapter03.pdf'> <class 'posix.DirEntry'>
data/webfiles-samples/pdf/Chapter03.pdf
<DirEntry 'Chapter03.pdf'> : f8b287ab8fb8dcf97417fdc6f19ec2ac 
digest2(Chapter03.pdf): 99561f13dffd3ad90c1fde3a26fc419d
<DirEntry 'PFCHEJ.pdf'> <class 'posix.DirEntry'>
data/webfiles-samples/pdf/PFCHEJ.pdf
<DirEntry 'PFCHEJ.pdf'> : cc8f5d1555056c7d9a8bd00d7b6fde37 
digest2(PFCHEJ.pdf): 729f7d1ffa927ce25494d08de53c257d
<DirEntry 'HR2021 commtext.pdf'> <class 'posix.DirEntry'>
data/webfiles-samples/pdf/HR2021 commtext.pdf
<DirEntry 'HR2021 commtext.pdf'> : 3c7ea953ba5ce0576ff4d86da9d935ed 
digest2(HR2021 comm

## Write the results to a CSV

Now that we can gather the information, let's write to a file:

In [55]:
import csv

In [56]:
## write to a CSV
# To do: Create a header row, write rows of file information, close the complete file

# set up the csv, create a header row
headers = [
    'index',
    'filename',
    'in_folder_path',
    'full_file_path',
    'size'
    ]

# write the information using csvwriter()
with open('file-manifest.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    for file in manifestInfo:
        print(file)
        writer.writerow(file)
    print('Wrote the file manifest')

[1, '.DS_Store', 'data/webfiles-samples', 'data/webfiles-samples/.DS_Store', 6148]
[2, 'web-files-small-metadata.csv', 'data/webfiles-samples', 'data/webfiles-samples/web-files-small-metadata.csv', 9069]
[3, 'vlwhcssc.asx', 'data/webfiles-samples/video', 'data/webfiles-samples/video/vlwhcssc.asx', 356]
[4, '04-04-21full.asf', 'data/webfiles-samples/video', 'data/webfiles-samples/video/04-04-21full.asf', 98]
[5, 'glmp_cig.EQ.wm.p20.t12z', 'data/webfiles-samples/video', 'data/webfiles-samples/video/glmp_cig.EQ.wm.p20.t12z', 8296]
[6, 'oct17cc.asx', 'data/webfiles-samples/video', 'data/webfiles-samples/video/oct17cc.asx', 106945]
[7, '01-1480.pdf', 'data/webfiles-samples/pdf', 'data/webfiles-samples/pdf/01-1480.pdf', 49088]
[8, 'file.pdf', 'data/webfiles-samples/pdf', 'data/webfiles-samples/pdf/file.pdf', 1538]
[9, 'Chapter03.pdf', 'data/webfiles-samples/pdf', 'data/webfiles-samples/pdf/Chapter03.pdf', 51919]
[10, 'PFCHEJ.pdf', 'data/webfiles-samples/pdf', 'data/webfiles-samples/pdf/PFCHE

## Reflection Activities

1. Write a script that uses `os.listdir()` for each of the directories in the `webfiles-samples` directory. You can put in the path names directly as variables, but you should use the `os.path.join()` function to create filepaths that do not depend on your inputting the exact filepath string, which will vary across operating systems.
1. Write a script that uses `os.scandir()` to check whether or not the entities in the director are files or directories. The script should output a count of files and a count of directories.
1. Examine the examples above that use `os.walk()`. What is the difference between this and the previous two functions? In some ways it lets you get deeper into the file structure, so please explain your observation in a sentence or two.  
1. Create a script that will print out an inventory of all the files in the assets folder `webfiles-samples`. The list should include the filename of the file, the directory path for the file, the full path to the file, and the file size. You may include any other information that you think is important. Call your script file `inventory_script.py`.
1. Extend the previous script: Create a script that will print out an inventory of all the files in the assets folder `webfiles-samples`. The inventory should be a CSV file, and it should include the filename of the file, the directory path for the file, the full path to the file, and the file size. You may include any other information that you think is important. Call your script file `inventory_script.py`.
1. Extend the above script, using the techniques demonstrated here, and add in a way to determine the file extension of the file, then add the extension to the CSV output? (Hint: you could split the filename string, right?)
1. Write a script that can walk through a series of directories and identify files based on their file extension. For example, perhaps you want to count the number of .pdf files or .jpg. Create file that can look for this information and then tally the files. Then, have the program output the list of filenames and filepaths in a CSV file. Call this file `extension_detector.py`. 

### Function exercises 

1. Building on the above examples, can you a) write functions that bundle code to ask for a directory? You could call this function `create_manifest_information` and it should be able to accept a path to a directory as an argument and return the manifestInfo list. And b) write a function that would accept the manifestInfo list as an argument and create a CSV? 
1. Write a script that creates a `master` and `derivative` directory within a subdirectory that has the file's name as its name. For example, if there are two files, one named `001.jpg` and `audition.wav`, there should be a directory named `001` and another named `audition`. Within these, there should be master and derivative folders. The original files should be in the `master` folder. Call this file `master_and_derivatives.py`.